In [1]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [2]:
'''
    1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и 
    "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут
'''

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

X = newsgroups.data
y = newsgroups.target

In [49]:
'''
    2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить
    TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию 
    из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной 
    из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент
    обучения, и поэтому можно ими пользоваться при обучении алгоритма.
'''

tdfv = TfidfVectorizer()
tdfv.fit_transform(X)

<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

In [50]:
'''
    3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром 
    (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для 
    KFold. В качестве меры качества используйте долю верных ответов (accuracy).
'''

grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)

gs.fit(tdfv.transform(X),y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [51]:
'''
    4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.
'''

clf1 = SVC(kernel='linear', random_state=241,C=10)
clf1.fit(tdfv.transform(X) , y)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [52]:
'''
    5.Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC).
    Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем 
    регистре, в лексикографическом порядке.
'''

words = tdfv.get_feature_names()
coef = pd.DataFrame(clf1.coef_.indices, clf1.coef_.data)

ans_word = []

Q = sorted(np.abs(clf1.coef_.data),reverse=True)

for i in Q[:10]:
    ans_word.append(words[coef[abs(coef.index) == i].values[0][0]])
    
for i in sorted(ans_word):
    print(i,end=',')

atheism,atheists,bible,god,keith,moon,nick,religion,sky,space,